Importar librerias

In [1]:
import json
import ast
import pandas as pd


Abrir Datasets

In [2]:
user_reviews = pd.read_csv(r'Datasets/reviews.csv')

In [3]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    59333 non-null  object 
 1   posted     59305 non-null  object 
 2   item_id    59305 non-null  float64
 3   helpful    59305 non-null  object 
 4   recommend  59305 non-null  object 
 5   review     59275 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.7+ MB


In [4]:
user_reviews.dtypes

user_id       object
posted        object
item_id      float64
helpful       object
recommend     object
review        object
dtype: object

In [5]:
datos_nulos= user_reviews.isnull().sum()
datos_nulos

user_id       0
posted       28
item_id      28
helpful      28
recommend    28
review       58
dtype: int64

In [6]:
#Eliminar los datos nulos
user_reviews= user_reviews.dropna(subset=['posted','review','helpful','item_id'])


In [7]:
#Verificamos que se haya aplicado
datos_nulos= user_reviews.isnull().sum()
datos_nulos

user_id      0
posted       0
item_id      0
helpful      0
recommend    0
review       0
dtype: int64

In [8]:
#cambiamos el tipo de dato de la columna "recommend" "item_id" 
user_reviews['recommend'] = user_reviews['recommend'].astype('bool')
user_reviews['item_id'] = user_reviews['item_id'].astype('int64')
user_reviews.dtypes

user_id      object
posted       object
item_id       int64
helpful      object
recommend      bool
review       object
dtype: object

In [9]:
user_reviews.head(5)

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Creamos la columna sentiment_analysis

Aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

Importamos liberias a utilizar

In [10]:
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [11]:
#Cambiamos el tipo de dato para el analisis de sentimiento pueda interpretarlo
user_reviews['review'] = user_reviews['review'].astype(str)

In [12]:
def analisis_sentimientos(var):
    if var is None:
        return 1 #Neutral
    analisis = TextBlob(var)
    polaridad = analisis.sentiment.polarity
    if polaridad >= 0.25:
        return 2 #Positivo
    elif polaridad <= -0.30:
        return 0 #Malo
    else:
        return 1 #Neutro

In [13]:
user_reviews["sentiment_analisis"]= user_reviews["review"].astype(str).apply(analisis_sentimientos)
user_reviews.head(3)

,user_id,posted,item_id,helpful,recommend,review,sentiment_analisis
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1


In [14]:
#modificamos la columna posted para que contenga solo el año posteado
user_reviews['posted'] = user_reviews['posted'].str.extract(r'Posted([\w\s\d,]+)') #se cambia la palabra Posted por espacio vacio
user_reviews['Posted'] = user_reviews['posted'].str.extract(r'(\d{4})').fillna('0') #extraigo el año y relleno los NAN por 0
user_reviews.drop(columns=['posted'])

,user_id,item_id,helpful,recommend,review,sentiment_analisis,Posted
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1,2011
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1,2011
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1,2014
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1,2013
...,...,...,...,...,...,...,...
59300,Fuckfhaisjnsnsjakaka,70,No ratings yet,True,a must have classic from steam definitely wort...,1,0
59301,3214213216,362890,No ratings yet,True,this game is a perfect remake of the original ...,1,0
59302,ChrisCoroner,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,1,0
59303,CaptainAmericaCw,730,No ratings yet,True,:D,2,0


In [15]:
user_reviews.to_csv(r'datasets/reviews_ETL.csv', index=False)